In [1]:
import pandas as pd
import glob
import os
from datetime import datetime
import re

# Função para formatar CNPJ
def formatar_cnpj(cnpj):
    if pd.isna(cnpj) or str(cnpj).strip() == '':
        return cnpj
    numeros = re.sub(r'\D', '', str(cnpj))
    if len(numeros) < 14:
        numeros = numeros.zfill(14)
    if len(numeros) == 14:
        return f"{numeros[:2]}.{numeros[2:5]}.{numeros[5:8]}/{numeros[8:12]}-{numeros[12:]}"
    else:
        return "CNPJ Inválido"

# Caminho da pasta com os arquivos Excel
caminho_pasta = r'C:\Users\Lucas.Cabral\Unilever\Canal RB - BASES DE PARA - TIME DE VENDAS\JULHO 25/*.xlsx'
arquivos_excel = glob.glob(caminho_pasta)

# Pasta de saída
pasta_saida = r'C:\Users\Lucas.Cabral\Unilever\PROJETO JBP - ROTA DA BELEZA - Documents\DE PARA - PBI\DE PARA VENDAS - BASES TRATADAS'
os.makedirs(pasta_saida, exist_ok=True)

# Limpa arquivos antigos da pasta de saída
for antigo in os.listdir(pasta_saida):
    caminho_antigo = os.path.join(pasta_saida, antigo)
    if os.path.isfile(caminho_antigo):
        os.remove(caminho_antigo)

# Data atual
data_hoje = datetime.today().strftime('%Y-%m-%d')

# Processa cada arquivo
for arquivo in arquivos_excel:
    try:
        df = pd.read_excel(arquivo, engine='openpyxl', skiprows=1)

        if df.empty:
            print(f"Arquivo {arquivo} está vazio após tratamento. Ignorado.")
            continue

        # Remove espaços dos nomes das colunas
        df.columns = df.columns.str.strip()

        # Corrige nome das colunas específicas
        col_b = 'CNPJ'
        col_e = 'CNPJ CENTRALIZADO'

        # Preenche coluna E com valor da coluna B se estiver vazia
        if col_b in df.columns and col_e in df.columns:
            df[col_e] = df.apply(lambda row: row[col_b] if pd.isna(row[col_e]) or str(row[col_e]).strip() == '' else row[col_e], axis=1)

        # Padroniza colunas de setor/região
        for col in df.columns:
            if 'setor' in col.lower() or 'região' in col.lower():
                df[col] = df[col].astype(str).str.strip().str.upper()
                df[col] = df[col].replace({
                    'SP1': 'SÃO PAULO 1',
                    'NORDESTE': 'NE',
                    'SUDESTE': 'SE'
                })

        # Converte colunas de data
        for col in df.columns:
            if 'data' in col.lower():
                df[col] = pd.to_datetime(df[col], errors='coerce')

        # Remove espaços de strings
        for col in df.select_dtypes(include='object').columns:
            df[col] = df[col].apply(lambda x: x if pd.isna(x) else str(x).strip())

        # Formata CNPJ, exceto na coluna 'É CNPJ COMPRADOR?'
        for col in df.columns:
            if 'cnpj' in col.lower() and col != 'É CNPJ COMPRADOR?':
                df[col] = df[col].apply(formatar_cnpj)

        # Remove duplicatas por Loja + Região
        if 'Loja' in df.columns and 'Região' in df.columns:
            df['Loja_Região'] = df['Loja'].astype(str) + '_' + df['Região'].astype(str)
            df = df.drop_duplicates(subset='Loja_Região')
            df = df.drop(columns='Loja_Região')

        # Salva arquivo tratado
        nome_arquivo = os.path.basename(arquivo)
        nome_tratado = f"{os.path.splitext(nome_arquivo)[0]}_tratado_{data_hoje}.xlsx"
        caminho_saida = os.path.join(pasta_saida, nome_tratado)
        df.to_excel(caminho_saida, index=False)

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {e}")

# Consolidação dos arquivos tratados
pasta_consolidada = os.path.join(pasta_saida, 'Arquivos consolidados')
os.makedirs(pasta_consolidada, exist_ok=True)

# Apagar arquivos anteriores com padrão 'Base_Consolidada_*.xlsx'
arquivos_consolidados_anteriores = glob.glob(os.path.join(pasta_consolidada, 'Base_Consolidada_*.xlsx'))
for arquivo_antigo in arquivos_consolidados_anteriores:
    try:
        os.remove(arquivo_antigo)
        print(f"Arquivo anterior removido: {arquivo_antigo}")
    except Exception as e:
        print(f"Erro ao remover {arquivo_antigo}: {e}")

arquivos_tratados = glob.glob(os.path.join(pasta_saida, '*.xlsx'))
dataframes = []

for arquivo in arquivos_tratados:
    try:
        df = pd.read_excel(arquivo, engine='openpyxl')
        if not df.empty:
            df['Arquivo_Origem'] = os.path.basename(arquivo)
            dataframes.append(df)
        else:
            print(f"Arquivo tratado {arquivo} está vazio. Ignorado.")
    except Exception as e:
        print(f"Erro ao ler o arquivo {arquivo}: {e}")

if dataframes:
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    nome_consolidado = f'Base_Consolidada_{data_hoje}.xlsx'
    caminho_consolidado = os.path.join(pasta_consolidada, nome_consolidado)
    df_consolidado.to_excel(caminho_consolidado, index=False)
    print(f"Arquivo consolidado salvo em: {caminho_consolidado}")
else:
    print("Nenhum arquivo válido para consolidar.")



Arquivo anterior removido: C:\Users\Lucas.Cabral\Unilever\PROJETO JBP - ROTA DA BELEZA - Documents\DE PARA - PBI\DE PARA VENDAS - BASES TRATADAS\Arquivos consolidados\Base_Consolidada_2025-08-12.xlsx
Arquivo consolidado salvo em: C:\Users\Lucas.Cabral\Unilever\PROJETO JBP - ROTA DA BELEZA - Documents\DE PARA - PBI\DE PARA VENDAS - BASES TRATADAS\Arquivos consolidados\Base_Consolidada_2025-08-18.xlsx
